In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime, timedelta

# --- 1단계: 모든 개별 이슈의 '원시 점수'를 계산하는 함수 (변경 없음) ---
def calculate_raw_metrics(news_df, social_df, period_start_date, period_end_date):
    """
    [명세서 기준 수정 완료]
    - V, G, A: 'period_end_date' 이전의 '전체' 뉴스 데이터를 사용
    - P: 'period_start_date' ~ 'period_end_date' 사이의 '기간 내' 뉴스 데이터만 사용
    - C, B: 'period_start_date' ~ 'period_end_date' 사이의 '기간 내' 소셜 데이터만 사용
    """
    news_full_history = news_df[news_df['timestamp'] <= period_end_date]
    news_in_period = news_df[(news_df['timestamp'] >= period_start_date) & (news_df['timestamp'] <= period_end_date)]
    social_in_period = social_df[(social_df['timestamp'] >= period_start_date) & (social_df['timestamp'] <= period_end_date)]

    if news_full_history.empty and social_in_period.empty:
        return pd.DataFrame()

    all_issue_ids = pd.concat([news_full_history['issue_id'], social_in_period['issue_id']]).unique()
    issue_metrics = {}

    for issue in all_issue_ids:
        # --- V, G, A: '전체 기간' 뉴스 데이터 기반 계산 ---
        news_issue_history = news_full_history[news_full_history['issue_id'] == issue].copy()
        v_score, g_score, a_score = 0.0, 0.0, 0.0
        if not news_issue_history.empty:
            half_life = 7.0
            news_issue_history['days_ago'] = (period_end_date - news_issue_history['timestamp']).dt.total_seconds() / (24 * 3600)
            news_issue_history['weight'] = np.exp(-np.log(2) * news_issue_history['days_ago'] / half_life)
            v_score = news_issue_history['weight'].sum()

            news_issue_history['week'] = news_issue_history['timestamp'].dt.to_period('W')
            weekly_volume = news_issue_history.groupby('week')['weight'].sum().to_timestamp().resample('W').sum().fillna(0)
            
            v_week_0 = weekly_volume.iloc[-1] if len(weekly_volume) > 0 else 0
            v_week_1 = weekly_volume.iloc[-2] if len(weekly_volume) > 1 else 0
            v_week_2 = weekly_volume.iloc[-3] if len(weekly_volume) > 2 else 0
            
            g_score = (v_week_0 - v_week_1) / v_week_1 if v_week_1 > 0 else v_week_0
            g_previous = (v_week_1 - v_week_2) / v_week_2 if v_week_2 > 0 else v_week_1
            a_score = g_score - g_previous

        # --- P: '기간 내' 뉴스 데이터 기반 계산 ---
        news_issue_in_period = news_in_period[news_in_period['issue_id'] == issue]
        p_score = 0.0
        if not news_issue_in_period.empty:
            total_days_in_period = (period_end_date - period_start_date).days + 1
            days_with_mentions = news_issue_in_period['timestamp'].dt.normalize().nunique()
            p_score = days_with_mentions / total_days_in_period if total_days_in_period > 0 else 0

        # --- C, B: '기간 내' 소셜 데이터 기반 계산 ---
        social_issue_in_period = social_in_period[social_in_period['issue_id'] == issue]
        c_score, b_score = 0.0, 0.0
        if not social_issue_in_period.empty:
            sentiment_counts = social_issue_in_period['label'].value_counts()
            pos_count = sentiment_counts.get('찬성_개정강화', 0) + sentiment_counts.get('찬성_폐지완화', 0)
            neg_count = sentiment_counts.get('반대_현상유지', 0)
            neu_count = sentiment_counts.get('중립', 0)
            
            total_polarized = pos_count + neg_count
            if total_polarized > 1:
                balance_factor = 1 - abs(pos_count - neg_count) / total_polarized
                c_score = total_polarized * balance_factor
            else:
                c_score = 0
            b_score = total_polarized + neg_count

        issue_metrics[issue] = {'V': v_score, 'P': p_score, 'G': g_score, 'A': a_score, 'C': c_score, 'B': b_score}

    return pd.DataFrame.from_dict(issue_metrics, orient='index')


# --- 2단계: '법안' 이슈를 그룹화하고 최종 TOP 5를 계산하는 함수 (변경 없음) ---
def analyze_legislation_top5(raw_metrics_df, legislation_mapping_df):
    if raw_metrics_df.empty or legislation_mapping_df.empty:
        return pd.DataFrame()

    merged_df = raw_metrics_df.merge(legislation_mapping_df, left_index=True, right_on='issue_id', how='inner')
    
    if merged_df.empty or 'legislation_id' not in merged_df.columns:
        return pd.DataFrame()
        
    agg_logic = {
        'V': 'sum', 'P': 'max', 'G': 'mean',
        'A': 'mean', 'C': 'sum', 'B': 'sum'
    }
    aggregated_metrics = merged_df.groupby('legislation_id').agg(agg_logic)

    if len(aggregated_metrics) == 0:
        return pd.DataFrame()

    scaler = MinMaxScaler()
    normalized_metrics = scaler.fit_transform(aggregated_metrics) if len(aggregated_metrics) > 1 else np.full_like(aggregated_metrics, 0.5, dtype=float)

    normalized_df = pd.DataFrame(normalized_metrics, columns=aggregated_metrics.columns, index=aggregated_metrics.index)
    
    weights = {'V': 0.2, 'P': 0.1, 'G': 0.15, 'C': 0.25, 'B': 0.2, 'A': 0.1}
    normalized_df['IIS'] = sum(normalized_df[col] * weights[col] for col in weights)
    
    def assign_grade(score):
        if score >= 0.7: return 'High'
        elif score >= 0.4: return 'Medium'
        else: return 'Low'
    normalized_df['Grade'] = normalized_df['IIS'].apply(assign_grade)

    return normalized_df.sort_values(by='IIS', ascending=False).head(5)

# --- [MAIN EXECUTION] ---
if __name__ == "__main__":
    # 1. 파일 경로 설정
    news_data_file = 'final_merged_data.csv'
    social_data_file = 'news_comments_all_감성분석완료.csv'
    legal_text_file = 'Law_total.csv' # [추가] 법률 내용 파일 경로
    
    try:
        print(f"'{news_data_file}' 파일을 불러오는 중...")
        news_raw_df = pd.read_csv(news_data_file, dtype={'date': str})
        
        print(f"'{social_data_file}' 파일을 불러오는 중...")
        social_raw_df = pd.read_csv(social_data_file, dtype={'comment_date': str})
    except FileNotFoundError:
        print("❌ 오류: CSV 파일을 찾을 수 없습니다. 예시 데이터로 실행합니다.")

    # 2. 데이터 전처리
    print("\n데이터 전처리를 시작합니다...")
    print(f"필터링 전 원본 뉴스 데이터 행 수: {len(news_raw_df)}")
    condition_to_exclude = (news_raw_df['mapped_law'] == '해당 없음') & (news_raw_df['mapped_article'] == '해당 없음')
    news_raw_df = news_raw_df[~condition_to_exclude].copy()
    print(f"'해당 없음' 법안을 제외한 유효 뉴스 데이터 행 수: {len(news_raw_df)}")

    news_df = news_raw_df.rename(columns={'cluster_event_name': 'issue_id', 'date': 'timestamp'})
    format_string = '%Y%m%d%H%M%S'
    news_df['timestamp'] = pd.to_datetime(news_df['timestamp'], format=format_string, errors='coerce')
    news_df.dropna(subset=['timestamp', 'issue_id', 'mapped_law', 'mapped_article'], inplace=True)

    social_df = social_raw_df.rename(columns={'cluster_event_name': 'issue_id', 'comment_date': 'timestamp'})
    social_df['timestamp'] = pd.to_datetime(social_df['timestamp'], format=format_string, errors='coerce')
    social_df.dropna(subset=['timestamp', 'issue_id', 'label'], inplace=True)

    # 3. '법안 식별자' 및 매핑 테이블 생성
    print("\n'법안 식별자' (legislation_id)를 생성합니다...")
    news_df['legislation_id'] = news_df['mapped_law'] + " " + news_df['mapped_article']
    legislation_mapping_df = news_df[['issue_id', 'legislation_id']].drop_duplicates()
    
    print("--- 생성된 법안 매핑 정보 (샘플) ---")
    print(legislation_mapping_df.head())

    # 4. 전체 기간 설정 및 분석 실행
    all_dates = pd.concat([news_df['timestamp'], social_df['timestamp']])
    if all_dates.empty:
        print("분석할 날짜 데이터가 없습니다.")
    else:
        period_start = all_dates.min()
        period_end = all_dates.max()

        print("\n\n" + "="*50)
        print(f"      전체 기간 최종 '법안' TOP 5 (기간: {period_start.date()} ~ {period_end.date()})")
        print("="*50)
        
        raw_metrics = calculate_raw_metrics(news_df, social_df, period_start, period_end)
        final_top_5 = analyze_legislation_top5(raw_metrics, legislation_mapping_df)
        
        if final_top_5.empty:
            print("분석할 '법안' 관련 데이터가 없습니다.")
        else:
            # --- [기능 추가 1] TOP 5 순위표 출력 ---
            print("--- [요약] TOP 5 순위표 ---")
            print(final_top_5)

            # --- [기능 추가 2] 법안별 상세 내용 및 관련 사건 조회 ---
            print("\n\n" + "="*70)
            print("      TOP 5 법안 상세 분석 (관련 사건 및 법률 내용)")
            print("="*70)

            try:
                legal_df = pd.read_csv(legal_text_file, encoding='utf-8-sig')
                legal_df['legislation_id'] = legal_df['법률명'].astype(str) + " " + legal_df['조 번호'].astype(str)
                legal_content_available = True
            except FileNotFoundError:
                print(f"⚠️ 경고: 법률 내용 파일('{legal_text_file}')을 찾을 수 없어, 법안 내용은 생략됩니다.")
                legal_content_available = False
            except KeyError as e:
                print(f"⚠️ 경고: 법률 내용 파일에 필요한 열({e})이 없습니다. 법안 내용은 생략됩니다.")
                legal_content_available = False

            top_5_ids = final_top_5.index.tolist()
            for rank, leg_id in enumerate(top_5_ids, 1):
                print(f"\n--- [ {rank}위 ] {leg_id} ---")
                
                # 관련 주요 사건 목록 출력
                associated_events = legislation_mapping_df[legislation_mapping_df['legislation_id'] == leg_id]['issue_id'].tolist()
                if associated_events:
                    print("▶ 관련 주요 사건:")
                    for event in associated_events:
                        print(f"  - {event}")
                else:
                    print("▶ 관련 주요 사건: (정보 없음)")

                # 법률 내용 출력
                if legal_content_available:
                    content_row = legal_df[legal_df['legislation_id'] == leg_id]
                    if not content_row.empty:
                        # '조제목'과 '조내용' 열이 있는지 확인 후 출력
                        if '조제목' in content_row.columns and pd.notna(content_row.iloc[0]['조제목']):
                            print(f"▶ 조제목: {content_row.iloc[0]['조제목']}")
                        if '조내용' in content_row.columns and pd.notna(content_row.iloc[0]['조내용']):
                            print(f"▶ 조내용:\n{content_row.iloc[0]['조내용']}")
                        else:
                             print("▶ 조내용: (내용 정보 없음)")
                    else:
                        print("▶ 조내용: (일치하는 법안 정보를 찾을 수 없음)")
                
                print("-" * 50)

'final_merged_data.csv' 파일을 불러오는 중...
'news_comments_all_감성분석완료.csv' 파일을 불러오는 중...

데이터 전처리를 시작합니다...
필터링 전 원본 뉴스 데이터 행 수: 27969
'해당 없음' 법안을 제외한 유효 뉴스 데이터 행 수: 25850

'법안 식별자' (legislation_id)를 생성합니다...
--- 생성된 법안 매핑 정보 (샘플) ---
                     issue_id               legislation_id
0      BDC 도입 자본시장법 개정안 통과 사건     자본시장과 금융투자업에 관한 법률 제81조 
12       박규리 전 연인 코인 사기 연루 의혹    자본시장과 금융투자업에 관한 법률 제178조 
83       상법 개정안 이사 충실의무 확대 논란     자본시장과 금융투자업에 관한 법률 제96조 
1428  이재용 삼성 무죄 판결 및 자본시장법 개정  자본시장과 금융투자업에 관한 법률 제165조의4 
1470       공매도 재개 및 무차입 방지 조치  자본시장과 금융투자업에 관한 법률 제180조의6 


      전체 기간 최종 '법안' TOP 5 (기간: 2025-01-01 ~ 2025-10-30)
--- [요약] TOP 5 순위표 ---
                                    V         P         G         A         C  \
legislation_id                                                                  
자본시장과 금융투자업에 관한 법률 제96조      0.022844  1.000000  0.137251  0.285807  1.000000   
자본시장과 금융투자업에 관한 법률 제178조의2   1.000000  0.823529  0.225300  0.298383  0.481576   
아동복지법 제17조  